### --- Day 5: Supply Stacks ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/5

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2022, 5);
WriteLines(inputLines, maxRows: 20);

Total lines: 512
Max line length: 35

[T]     [Q]             [S]        
[R]     [M]             [L] [V] [G]
[D] [V] [V]             [Q] [N] [C]
[H] [T] [S] [C]         [V] [D] [Z]
[Q] [J] [D] [M]     [Z] [C] [M] [F]
[N] [B] [H] [N] [B] [W] [N] [J] [M]
[P] [G] [R] [Z] [Z] [C] [Z] [G] [P]
[B] [W] [N] [P] [D] [V] [G] [L] [T]
 1   2   3   4   5   6   7   8   9 

move 5 from 4 to 9
move 3 from 5 to 1
move 12 from 9 to 6
move 1 from 6 to 9
move 3 from 2 to 8
move 6 from 3 to 9
move 2 from 2 to 9
move 2 from 3 to 5
move 9 from 8 to 1
move 1 from 6 to 9


In [4]:
string[] testInputLines = [
    "    [D]    ",
    "[N] [C]    ",
    "[Z] [M] [P]",
    " 1   2   3 ",
    "",
    "move 1 from 2 to 1",
    "move 3 from 1 to 3",
    "move 2 from 2 to 1",
    "move 1 from 1 to 2",
];

In [5]:
using ColStack = SCG.Dictionary<int, SCG.List<char>>;

In [6]:
(ColStack colStacks, string[] moves) ParseStacks(string[] inputLines)
{
    var (stack, moves) = inputLines.SeparateBy(line => line is "").ToArray();
    
    var alphabet = Alphabet.ToUpper().ToHashSet();

    ColStack stacks = new();
    foreach (var (row, col, ch) in getAll(stack))
    {
        var colStack = stacks[col] = stacks.GetValueOrDefault(col, new());
        colStack.Insert(0, ch);
    }

    return (stacks, moves);

    // Helper method: get every row/col that has a char
    IEnumerable<(int row, int col, char ch)> getAll(string[] stackLines)
    {
        foreach ((var row, var line) in stackLines.Index())
        foreach ((var col, var ch) in line[1..^1].Index())
        {
            if (alphabet.Contains(ch))
            {
                yield return (row, 1 + col / 4, ch);
            }
        }
    }
}

In [7]:
void DoMove(ColStack colStacks, string inputLine)
{
    var (count, from, to) = inputLine.ParseAll(@"\d+").Select(int.Parse).ToArray();
    
    // NB: the move is individual pick / drop of N items, not picking N items at
    // once
    colStacks[to].AddRange(Enumerable.Reverse(colStacks[from][^count..]));
    colStacks[from] = colStacks[from][..^count];
}

void DoAllMoves(ColStack colStacks, string[] inputLines, Action<ColStack, string> moveFunc = null)
{
    moveFunc ??= DoMove; // Spoiler alert, this changes in part 2

    foreach (var inputLine in inputLines)
    {
        moveFunc(colStacks, inputLine);
    }
}


In [8]:
string DoAllMoves(string[] inputLines, Action<ColStack, string> moveFunc = null)
{
    var (colStacks, moves) = ParseStacks(inputLines);
    DoAllMoves(colStacks, moves, moveFunc);
    char[] resultChars = new char[colStacks.Count];
    foreach (var (col, stack) in colStacks)
    {
        resultChars[col-1] = stack[^1];
    }
    return new(resultChars);
}

In [9]:
// The Elves just need to know which crate will end up on top of each stack; in
// this example, the top crates are C in stack 1, M in stack 2, and Z in stack 3,
// so you should combine these together and give the Elves the message CMZ.

var testAnswer = DoAllMoves(testInputLines);
Console.WriteLine(testAnswer);

In [10]:
// After the rearrangement procedure completes, what crate ends up on top of
// each stack?

var part1Answer = DoAllMoves(inputLines);
Console.WriteLine(part1Answer);


In [11]:
// ZBDRNPMVH is correct!
Ensure("ZBDRNPMVH", part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/5

In [13]:
void DoMove2(ColStack colStacks, string inputLine)
{
    var (count, from, to) = inputLine.ParseAll(@"\d+").Select(int.Parse).ToArray();

    // NB: this time we pick up all the items at once!
    colStacks[to].AddRange(colStacks[from][^count..]);
    colStacks[from] = colStacks[from][..^count];
}

In [14]:
// In this example, the CrateMover 9001 has put the crates in a totally different order: MCD.

var part2TestAnswer = DoAllMoves(testInputLines, moveFunc: DoMove2);
Console.WriteLine(part2TestAnswer);

In [15]:
// After the rearrangement procedure completes, what crate ends up on top of each stack?

var part2Answer = DoAllMoves(inputLines, moveFunc: DoMove2);
Console.WriteLine(part2Answer);

In [16]:
// WDLPFNNNB is correct!
Ensure("WDLPFNNNB", part2Answer);